In [56]:
import forallpeople as si
import numpy as np
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

si.environment("default")

display(
    HTML(
        """
<style>
.calculator-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 12px 16px;
    border-radius: 8px;
    margin-bottom: 12px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.section-header {
    color: #667eea;
    border-bottom: 2px solid #667eea;
    padding-bottom: 4px;
    margin-top: 12px;
    margin-bottom: 8px;
    font-weight: 600;
    font-size: 1.1em;
}
.param-line {
    padding: 2px 0;
    padding-left: 20px;
    line-height: 1.4;
    font-size: 0.95em;
    max-width: 100%;
    overflow-wrap: break-word;
    word-wrap: break-word;
}
.slider-label {
    font-weight: 600;
    color: #333;
    font-size: 0.95em;
    min-width: 90px;
    display: inline-block;
    padding-right: 0px;
}
</style>
"""
    )
)


def format_si_value(value):
    """Format SI unit values to remove trailing zeros and format nicely"""
    if hasattr(value, "__str__"):
        # Get string representation
        str_val = str(value)
        # Split number and unit
        parts = str_val.split()
        if len(parts) == 2:
            num_str, unit = parts
            # Convert to float and back to remove trailing zeros
            num = float(num_str)
            # If it's effectively an integer, format without decimals
            if num == int(num):
                return f"{int(num)} {unit}"
            else:
                # Remove trailing zeros from decimal
                formatted = f"{num:g}"
                return f"{formatted} {unit}"
    return str(value)


def create_slider(
    label_html,
    value,
    min_val,
    max_val,
    step,
    is_int=False,
    readout_format=None,
):
    """Create a slider with label"""
    label_widget = widgets.HTML(
        f'<div class="slider-label">{label_html}</div>'
    )
    slider_class = widgets.IntSlider if is_int else widgets.FloatSlider
    slider_args = {
        "value": value,
        "min": min_val,
        "max": max_val,
        "step": step,
        "continuous_update": False,
        "style": {"handle_color": "#20c997"},
        "layout": widgets.Layout(width="auto", flex="1"),
    }
    if readout_format:
        slider_args["readout_format"] = readout_format
    slider = slider_class(**slider_args)
    return label_widget, slider


def create_compact_slider_row(label_widget, slider):
    """Create a compact HBox for slider with reduced spacing"""
    return widgets.HBox(
        [label_widget, slider],
        layout=widgets.Layout(margin="0px 0px 0px 0px", padding="0px"),
    )


# Create all sliders using the helper function
eta_label, eta_slider = create_slider("η:", 0.9, 0.5, 1.0, 0.01)
n_label, n_slider = create_slider("n:", 4, 1, 4, 1, is_int=True)
alpha_B_label, alpha_B_slider = create_slider(
    "&alpha;<sub>B</sub>:", 0.7, 0.1, 0.9, 0.05
)
p_backup_label, p_backup_slider = create_slider(
    "P<sub>BACKUP</sub> (W):", 25, 1, 100, 1
)
t_backup_label, t_backup_slider = create_slider(
    "t<sub>BACKUP</sub> (s):", 1.8, 0.1, 60, 0.1
)
v_cell_max_label, v_cell_max_slider = create_slider(
    "V<sub>CELL(MAX)</sub> (V):", 2.5, 1.0, 5.0, 0.1
)
r_snsc_label, r_snsc_slider = create_slider(
    "R<sub>SNSC</sub> (Ω):", 0.006, 0.001, 0.02, 0.001, readout_format=".4f"
)
esr_eol_label, esr_eol_slider = create_slider(
    "ESR<sub>EOL</sub> (Ω):", 0.064, 0.001, 0.2, 0.001, readout_format=".4f"
)
c_eol_label, c_eol_slider = create_slider(
    "C<sub>EOL</sub> (F):", 7, 1, 300, 1
)

input_params_output = widgets.Output()
input_values_output = widgets.Output()
output_results = widgets.Output()


def display_input_params_title():
    """Display input parameters title only"""
    with input_params_output:
        clear_output(wait=True)
        display(HTML('<div class="section-header">📊 Input Parameters</div>'))


def display_input_values(
    eta, n, alpha_B, p_backup, t_BACKUP, v_cell_max, r_snsc, esr_eol, c_eol
):
    """Display input parameter values"""
    with input_values_output:
        clear_output(wait=True)

        # Convert to SI units
        p_backup_si = p_backup * si.W
        t_BACKUP_si = t_BACKUP * si.s
        v_cell_max_si = v_cell_max * si.V
        r_snsc_si = r_snsc * si.Ohm
        esr_eol_si = esr_eol * si.Ohm
        c_eol_si = c_eol * si.F

        # Display input parameter values in a 5-column grid
        params_html = f"""
        <div style="display: grid; 
                    grid-template-columns: repeat(5, 1fr); 
                    gap: 4px 8px; max-width: 100%;">
            <div class="param-line">η = {eta}</div>
            <div class="param-line">n = {n}</div>
            <div class="param-line">&alpha;<sub>B</sub> = 
                {alpha_B}</div>
            <div class="param-line">P<sub>BACKUP</sub> = 
                {format_si_value(p_backup_si)}</div>
            <div class="param-line">t<sub>BACKUP</sub> = 
                {format_si_value(t_BACKUP_si)}</div>
            <div class="param-line">V<sub>CELL(MAX)</sub> = 
                {format_si_value(v_cell_max_si)}</div>
            <div class="param-line">R<sub>SNSC</sub> = 
                {format_si_value(r_snsc_si)}</div>
            <div class="param-line">ESR<sub>EOL</sub> = 
                {format_si_value(esr_eol_si)}</div>
            <div class="param-line">C<sub>EOL</sub> = 
                {format_si_value(c_eol_si)}</div>
        </div>
        """
        display(HTML(params_html))


def calculate_backup_time(
    cap_si, esr_si, eta, n, p_backup_si, v_cell_max_si, i_peak
):
    """Calculate backup time for given capacitor parameters"""
    # Calculate v_stk_min
    v_stk_min_power = np.sqrt(4 * esr_si * n * p_backup_si / eta)
    v_stk_min_current = (p_backup_si / (eta * i_peak)) + n * esr_si * i_peak
    v_stk_min_calc = max(v_stk_min_power, v_stk_min_current)

    # Calculate gamma values
    gama_max_calc = 1 + np.sqrt(
        1
        - (
            (4 * n * esr_si * p_backup_si)
            / (eta * np.power(n * v_cell_max_si, 2))
        )
    )
    gama_min_calc = 1 + np.sqrt(
        1
        - (
            (4 * n * esr_si * p_backup_si)
            / (eta * np.power(v_stk_min_calc, 2))
        )
    )

    # Calculate v_loss_square
    v_loss_sq_calc = ((4 * n * esr_si * p_backup_si) / eta) * np.log(
        (gama_max_calc * n * v_cell_max_si) / (gama_min_calc * v_stk_min_calc)
    )

    # Calculate backup time
    t_backup_calc = (
        eta
        * (cap_si / n)
        / (4 * p_backup_si)
        * (
            gama_max_calc * (n * v_cell_max_si) ** 2
            - gama_min_calc * v_stk_min_calc**2
            - v_loss_sq_calc
        )
    )

    return t_backup_calc


def calculate_and_display_results(
    eta, n, alpha_B, p_backup, t_BACKUP, v_cell_max, r_snsc, esr_eol, c_eol
):
    """Perform calculations and display results"""
    # Update input parameters title
    display_input_params_title()

    # Update input values
    display_input_values(
        eta,
        n,
        alpha_B,
        p_backup,
        t_BACKUP,
        v_cell_max,
        r_snsc,
        esr_eol,
        c_eol,
    )

    with output_results:
        clear_output(wait=True)

        # Convert to SI units
        p_backup_si = p_backup * si.W
        t_BACKUP_si = t_BACKUP * si.s
        v_cell_max_si = v_cell_max * si.V
        r_snsc_si = r_snsc * si.Ohm
        esr_eol_si = esr_eol * si.Ohm
        c_eol_si = c_eol * si.F

        # Calculations - displayed in 5-column grid
        display(
            HTML('<div class="section-header">🔬 Calculated Results</div>')
        )

        c_eol = (
            (4 * p_backup_si * t_BACKUP_si)
            / (n * eta * np.power(v_cell_max_si, 2))
            * np.power(
                alpha_B
                + np.sqrt(alpha_B)
                - (1 - alpha_B)
                * np.log(
                    (1 + np.sqrt(alpha_B)) / np.sqrt(1 - np.sqrt(alpha_B))
                ),
                -1,
            )
        )

        esr_eol = (
            eta
            * (1 - alpha_B)
            * n
            * np.power(v_cell_max_si, 2)
            / (4 * p_backup_si)
        )

        v_stk_min__at_max_power = np.sqrt(
            4 * esr_eol_si * n * p_backup_si / eta
        )

        i_peak = 0.058 * si.V / r_snsc_si

        v_stk_min__at_current_limit = (
            p_backup_si / (eta * i_peak)
        ) + n * esr_eol_si * i_peak

        v_stk_min = max(v_stk_min__at_max_power, v_stk_min__at_current_limit)

        gama_max = round(
            1
            + np.sqrt(
                1
                - (
                    (4 * n * esr_eol_si * p_backup_si)
                    / (eta * np.power(n * v_cell_max_si, 2))
                )
            ),
            4,
        )

        gama_min = round(
            1
            + np.sqrt(
                1
                - (
                    (4 * n * esr_eol_si * p_backup_si)
                    / (eta * np.power(v_stk_min, 2))
                )
            ),
            4,
        )

        v_loss_square = round(
            ((4 * n * esr_eol_si * p_backup_si) / eta)
            * np.log((gama_max * n * v_cell_max_si) / (gama_min * v_stk_min)),
            4,
        )

        t_backup_eol = (
            eta
            * (c_eol_si / n)
            / (4 * p_backup_si)
            * (
                gama_max * (n * v_cell_max_si) ** 2
                - gama_min * v_stk_min**2
                - v_loss_square
            )
        )

        results_html = f"""
        <div style="display: grid; 
                    grid-template-columns: repeat(5, 1fr); 
                    gap: 4px 8px; max-width: 100%;">
            <div class="param-line"><strong>C<sub>EOL</sub>
                </strong> = {format_si_value(c_eol)}</div>
            <div class="param-line"><strong>ESR<sub>EOL</sub>
                </strong> = {format_si_value(esr_eol)}</div>
            <div class="param-line"><strong>I<sub>PEAK</sub>
                </strong> = {format_si_value(i_peak)}</div>
            <div class="param-line"><strong>
                V<sub>STK_MIN_max_power</sub></strong> = 
                {format_si_value(v_stk_min__at_max_power)}</div>
            <div class="param-line"><strong>
                V<sub>STK_MIN_current_limit</sub></strong> = 
                {format_si_value(v_stk_min__at_current_limit)}</div>
            <div class="param-line"><strong>V<sub>STK_MIN</sub>
                </strong> = {format_si_value(v_stk_min)}</div>
            <div class="param-line"><strong>
                &gamma;<sub>MAX</sub></strong> = {gama_max}</div>
            <div class="param-line"><strong>
                &gamma;<sub>MIN</sub></strong> = {gama_min}</div>
            <div class="param-line"><strong>
                V&sup2;<sub>LOSS</sub></strong> = 
                {v_loss_square}</div>
            <div class="param-line"><strong>t<sub>BACKUP</sub>
                </strong> = {format_si_value(t_backup_eol)}</div>
        </div>
        """
        display(HTML(results_html))

        # EOL Note
        eol_note = """
        <div style="background: linear-gradient(135deg, 
                    #ffeaa7 0%, #fdcb6e 100%); 
                    border-radius: 12px; 
                    padding: 20px 24px; 
                    margin: 20px 0; 
                    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
                    border: 2px solid #fdcb6e;">
            <div style="display: flex; align-items: center; 
                        margin-bottom: 12px;">
                <span style="font-size: 2em; 
                             margin-right: 12px;">💡</span>
                <strong style="color: #2d3436; 
                               font-size: 1.2em;">
                    End of Life (EOL) - What Does It Mean?
                </strong>
            </div>
            <p style="margin: 0; color: #2d3436; 
                      line-height: 1.6; font-size: 0.95em;">
                A supercapacitor reaches its <strong>End of Life 
                (EOL)</strong> when it experiences a 
                <strong style="color: #d63031;">20%
                decrease in capacitance</strong> and/or a 
                <strong style="color: #d63031;">doubling of 
                Equivalent Series Resistance (ESR)</strong> from 
                initial values.
            </p>
        </div>
        """
        display(HTML(eol_note))

        # Backup Time Table
        display(
            HTML('<div class="section-header">⏱️ Backup Time Analysis</div>')
        )

        # Define capacitance and ESR value pairs (INITIAL values)
        cap_esr_pairs = [
            (10, 0.034),
            (15, 0.03),
            (20, 0.03),
            (50, 0.024),
            (100, 0.001),
            (300, 0.0045),
        ]

        # Build table HTML with 5 columns (one per capacitor)
        table_html = """
        <table style="width: 100%; border-collapse: collapse; 
                      margin-top: 10px; font-size: 0.9em;">
            <thead>
                <tr style="background-color: #667eea; 
                           color: white;">
        """

        # Header for each capacitor column
        for i in range(len(cap_esr_pairs)):
            table_html += (
                f'<th style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"Capacitor {i + 1}</th>"
            )

        table_html += """
                </tr>
            </thead>
            <tbody>
        """

        # Row 1: Power values
        table_html += '<tr style="background-color: #e8f4f8;">'
        for _ in enumerate(cap_esr_pairs):
            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>P<sub>BACKUP</sub>:</strong> "
                f"{int(p_backup)} W</td>"
            )
        table_html += "</tr>"

        # Row 2: Backup time from slider
        table_html += '<tr style="background-color: #fff3cd;">'
        for _ in enumerate(cap_esr_pairs):
            # Format t_BACKUP without trailing zeros
            t_backup_formatted = f"{t_BACKUP:g}"
            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>t<sub>BACKUP</sub>:</strong> "
                f"{t_backup_formatted} s</td>"
            )
        table_html += "</tr>"

        # Row 3: Capacitance values (Initial and EOL)
        table_html += '<tr style="background-color: #f8f9fa;">'
        for cap_initial, _ in cap_esr_pairs:
            cap_eol = cap_initial * 0.8  # C_EOL = C_initial * 0.8
            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>C </strong>(C<sub>EOL</sub>):<br>"
                f"<strong>{format_si_value(cap_initial * si.F)} </strong>"
                f"({format_si_value(cap_eol * si.F)})</td>"
            )
        table_html += "</tr>"

        # Row 4: ESR values (Initial and EOL)
        table_html += '<tr style="background-color: #ffffff;">'
        for _, esr_initial in cap_esr_pairs:
            esr_eol = esr_initial * 2  # ESR_EOL = ESR_initial * 2
            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>ESR </strong>(ESR<sub>EOL</sub>):</strong><br>"
                f"<strong>{format_si_value(esr_initial * si.Ohm)} </strong>"
                f"({format_si_value(esr_eol * si.Ohm)})</td>"
            )
        table_html += "</tr>"

        # Row 5: Initial backup time values
        table_html += '<tr style="background-color: #d4edda;">'
        for cap_initial, esr_initial in cap_esr_pairs:
            cap_si = cap_initial * si.F
            esr_si = esr_initial * si.Ohm

            # Calculate backup time for initial conditions
            t_backup_calc = calculate_backup_time(
                cap_si, esr_si, eta, n, p_backup_si, v_cell_max_si, i_peak
            )

            # Handle both SI Physical objects and numpy floats
            if hasattr(t_backup_calc, "value"):
                time_value = float(t_backup_calc.value)
            else:
                time_value = float(t_backup_calc)

            time_str = (
                '<span style="color: red;">N/A</span>'
                if time_value < 0 or np.isnan(time_value)
                else f"{time_value:.2f} s"
            )

            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>t<sub>BACKUP</sub>:</strong> "
                f"{time_str}</td>"
            )

        table_html += "</tr>"

        # Row 6: EOL backup time values
        table_html += '<tr style="background-color: #f8d7da;">'
        for cap_initial, esr_initial in cap_esr_pairs:
            cap_eol = cap_initial * 0.8  # C_EOL = C_initial * 0.8
            esr_eol = esr_initial * 2  # ESR_EOL = ESR_initial * 2

            cap_si = cap_eol * si.F
            esr_si = esr_eol * si.Ohm

            # Calculate backup time for EOL conditions
            t_backup_calc = calculate_backup_time(
                cap_si, esr_si, eta, n, p_backup_si, v_cell_max_si, i_peak
            )

            # Handle both SI Physical objects and numpy floats
            if hasattr(t_backup_calc, "value"):
                time_value = float(t_backup_calc.value)
            else:
                time_value = float(t_backup_calc)

            time_str = (
                '<span style="color: red;">N/A</span>'
                if time_value < 0 or np.isnan(time_value)
                else f"{time_value:.2f} s"
            )

            table_html += (
                f'<td style="border: 1px solid #ddd; '
                f'padding: 8px; text-align: center;">'
                f"<strong>t<sub>BACKUP_EOL</sub>:</strong> "
                f"{time_str}</td>"
            )

        table_html += "</tr>"

        table_html += """
            </tbody>
        </table>
        """

        display(HTML(table_html))


# Create interactive output
interactive_calc = widgets.interactive_output(
    calculate_and_display_results,
    {
        "eta": eta_slider,
        "n": n_slider,
        "alpha_B": alpha_B_slider,
        "p_backup": p_backup_slider,
        "t_BACKUP": t_backup_slider,
        "v_cell_max": v_cell_max_slider,
        "r_snsc": r_snsc_slider,
        "esr_eol": esr_eol_slider,
        "c_eol": c_eol_slider,
    },
)

# Display all widgets
display(
    widgets.VBox(
        [
            widgets.HTML(
                '<div class="calculator-header">'
                '<h2 style="margin:0; font-size:1.3em;">'
                "⚡ LTC3350 Interactive Calculator</h2></div>"
            ),
            input_params_output,
            create_compact_slider_row(eta_label, eta_slider),
            create_compact_slider_row(n_label, n_slider),
            create_compact_slider_row(alpha_B_label, alpha_B_slider),
            create_compact_slider_row(p_backup_label, p_backup_slider),
            create_compact_slider_row(t_backup_label, t_backup_slider),
            create_compact_slider_row(v_cell_max_label, v_cell_max_slider),
            create_compact_slider_row(r_snsc_label, r_snsc_slider),
            input_values_output,
            create_compact_slider_row(esr_eol_label, esr_eol_slider),
            create_compact_slider_row(c_eol_label, c_eol_slider),
            output_results,
        ],
        layout=widgets.Layout(
            padding="10px",
            width="100%",
            max_width="100%",
            align_items="stretch",
        ),
    )
)

# Trigger initial calculation
calculate_and_display_results(
    eta_slider.value,
    n_slider.value,
    alpha_B_slider.value,
    p_backup_slider.value,
    t_backup_slider.value,
    v_cell_max_slider.value,
    r_snsc_slider.value,
    esr_eol_slider.value,
    c_eol_slider.value,
)
